<a href="https://colab.research.google.com/github/hwangjunseong/my-first-website/blob/main/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9Dteam_project4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

사용할 라이브러리

In [ ]:
#!pip install catboost
!pip install lightgbm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import numpy.linalg as lin
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
#from catboost import CatBoostClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
import lightgbm as lgb

데이터 모으기 , 종속변수 벡터와 독립변수 벡터를 분리하기

In [ ]:
train = pd.read_csv("/content/fashion-mnist_train.csv")
test = pd.read_csv("/content/fashion-mnist_test.csv")



X_train = train.iloc[:,1:].values
y_train = train.iloc[:,0].values
X_test= test.iloc[:,1:].values
y_test=test.iloc[:,0].values

#y_train = np.array(y_train).ravel()#2차원 배열을 1차원 배열로 변경
#y_test = np.array(y_test).ravel()


pca를 진행하기전에 표준화 보통 [-3,3]에 값이 위치, 또는 정규화[0, 1] 둘 중 하나 선택

In [ ]:

#scaler_minmax=MinMaxScaler()
#X_train_std = scaler_minmax.fit_transform(X_train)
#X_test_std = scaler_minmax.transform(X_test)

In [ ]:
#입렦값으로 2차원 배열을 요구한다

scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_train_std = scaler_X.fit_transform(X_train)
X_test_std = scaler_X.transform(X_test)
#y_train = scaler_y.fit_transform(y_train)
#print(np.mean(X_train_std))

PCA

In [ ]:

# PCA 모델 생성
pca = PCA(n_components=2)
X_train_reduced = pca.fit_transform(X_train_std)
X_test_reduced = pca.transform(X_test_std)
#print(X_train_reduced)

In [ ]:
def plot_scree(X, max_dim):
    pca = PCA(n_components=max_dim)
    pca.fit(X)
    eigenvalues = pca.explained_variance_
    plt.plot(np.arange(1, max_dim+1), eigenvalues, 'o-', linewidth=2)
    plt.xlabel('Number of components')
    plt.ylabel('Eigenvalue')
    plt.title('Scree Plot')
    plt.show()

In [ ]:
#plot_scree(X_train_reduced, 80)

모델 선택 => k폴더 교차검증,그리디 서치=> 최적의 parameter찾기,  앙상블


In [ ]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier_svr, X = X_train_reduced, y = y_train, cv = 10)
printf("accuracy : {:.2f} %".format(accuracies.mean()*100))
printf("표준편차: {: .2f} %".format(accuracies.std()*100))

In [ ]:
param_grid = {

    'criterion': ['gini', 'entropy']  # 분할 품질을 측정하는 기준
}
grid_search=GridSearchCV(RandomForestClassifier(), param_grid, cv=10, return_train_score=True)
grid_search.fit(X_train_reduced,y_train) #여기서 y_train이 1차원 배열이여야함
rf = grid_search.best_params_
print("Best Parameter: {}".format(grid_search.best_params_))
print("Best Cross-validity Score: {:.3f}".format(grid_search.best_score_))

Best Parameter: {'criterion': 'entropy'}
Best Cross-validity Score: 0.504


In [ ]:
#parameters = [{'C': [0.25, 0.5, 0.75, 1], 'kernel': ['linear']}, #C가 낮을수록 정규화가 강해진다
#              {'C': [0.25, 0.5, 0.75, 1], 'kernel': ['rbf'], 'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}] #gamma는 커널의 공식 계수 -> gamma커널은 rbf에만 적용가능 linear에는 불가능
parameter_grid = {
    'C' : [0.25, 0.5, 0.75, 1]
}

#svm_model = SVC(kernel = 'rbf', probability = True)
grid_search = GridSearchCV(estimator = SVC(kernel = 'rbf', probability = True),
                           param_grid = parameter_grid,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1) # n_jobs : 머신에서 모든 프로세서가 사용된다

grid_search.fit(X_train_reduced, y_train)
svr = grid_search.best_params_
print("Best Parameter: {}".format(grid_search.best_params_))
print("Best Cross-validity Score: {:.3f}".format(grid_search.best_score_))

In [ ]:
#'max_iter': 100
#{'max_depth': 3}
#{'learning_rate': 0.01}
#{'max_leaf_nodes': 15}

In [ ]:
parameter_grid = {
    'learning_rate': [0.01, 0.05, 0.1],

}

hist_gb_model = HistGradientBoostingClassifier(random_state=42)
grid_search=GridSearchCV(hist_gb_model, parameter_grid, cv=10, return_train_score=True)
grid_search.fit(X_train_reduced, y_train)
hgm = grid_search.best_params_
# 데이터 평가
print("Best Parameter: {}".format(grid_search.best_params_))
print("Best Cross-validity Score: {:.3f}".format(grid_search.best_score_))

In [ ]:



# 모델
#classfier_cb = CatBoostClassifier(custom_metric = 'MultiClass')
#grid_search=GridSearchCV(classfier_cb, parameter_grid, cv=10, return_train_score=True)
#grid_search.fit(X_train_reduced, y_train)
#cbc = grid_search.best_params_
# 데이터 평가
#print("Best Parameter: {}".format(grid_search.best_params_))
#print("Best Cross-validity Score: {:.3f}".format(grid_search.best_score_))

In [ ]:
parameter_grid = {
   'gamma': [0, 0.1, 0.2],

}

xgb_model = XGBClassifier()

grid_search_xgb=GridSearchCV(xgb_model, parameter_grid, cv=10, return_train_score=True)
grid_search_xgb.fit(X_train_reduced, y_train)
xm = grid_search_xgb.best_params_
# 데이터 평가
print("Best Parameter: {}".format(grid_search.best_params_))
print("Best Cross-validity Score: {:.3f}".format(grid_search.best_score_))

In [ ]:
 #'max_depth': [3, 5, 7],  # 트리의 최대 깊이
 #   'num_leaves': [15, 31, 63],  # 리프 노드 수
 #   'subsample': [0.8, 1.0],  # 데이터 샘플링 비율
 #   'colsample_bytree': [0.8, 1.0],  # 트리마다 사용할 특성의 비율
 #   'reg_alpha': [0.0, 0.1, 0.5],  # L1 정규화 강도
 #   'reg_lambda': [0.0, 0.1, 0.5],  # L2 정규화 강도
 #   'min_child_samples': [5, 10, 20],

In [ ]:
parameter_grid = {
    'n_estimators': [50, 100, 200],

}
lgb_model = lgb.LGBMClassifier()
#{'n_estimators': 200
grid_search=GridSearchCV(lgb_model, parameter_grid, cv=10, return_train_score=True)
grid_search.fit(X_train_reduced, y_train)
lm = grid_search.best_params_
# 데이터 평가
print("Best Parameter: {}".format(grid_search.best_params_))
print("Best Cross-validity Score: {:.3f}".format(grid_search.best_score_))

In [ ]:
svm_model = SVC(kernel = 'rbf', probability = True, C= svr['C'], random_state = 42)
classifier_xgb = XGBClassifier(gamma = xm['gamma'], random_state = 42)
lgb_model = lgb.LGBMClassifier(n_estimators = lm['n_estimators'], random_state = 42)
hist_gb_model = HistGradientBoostingClassifier(learning_rate = hgm['learning_rate'],max_iter=100, random_state=42)
rf_model = RandomForestClassifier(criterion = rf['criterion'], random_state=42)

model = VotingClassifier(
    estimators=[ ('rf', rf_model),('hist_gb', hist_gb_model),('lm', lgb_model), ('cx', classifier_xgb), ('svc', svm_model)], voting='soft')
model.fit(X_train_reduced, y_train)

In [ ]:
X_test_reduced.shape

test데이터로 예측

In [ ]:
#np.array(X_test_reduced).ravel()

y_pred = (model.predict(X_test_reduced))
print(X_test_reduced)
print(y_pred)

혼동행렬 사용하여 정확도 예측

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

예측된 클래스 값을 testResult.txt 파일에 저장

In [ ]:

with open("testResult.txt", "w") as f:
    for index, prediction in enumerate(y_pred):
      index_str = f"{index:05d}" #5자리의 길이로 표시하고, 빈 자리는 0으로 채우는 문자열을 생성
      result = f"{index_str} {int(prediction)}"
      f.write(result + "\n")


In [ ]:
with open("testResult.txt", "r") as f:
    content = f.read()

print(content)